In [1]:
from sklearn import set_config
from sklearn.model_selection import (
    GridSearchCV,
    KFold,
    RandomizedSearchCV,
    train_test_split,
)
from sklearn.pipeline import Pipeline
from scipy.stats import loguniform

from skfolio import RatioMeasure, RiskMeasure
from skfolio.datasets import load_factors_dataset, load_sp500_dataset
from skfolio.model_selection import (
    CombinatorialPurgedCV,
    WalkForward,
    cross_val_predict,
)
from skfolio.moments import (
    DenoiseCovariance,
    DetoneCovariance,
    EWMu,
    GerberCovariance,
    ShrunkMu,
)
from skfolio.optimization import (
    MeanRisk,
    NestedClustersOptimization,
    ObjectiveFunction,
    RiskBudgeting,
)
from skfolio.pre_selection import SelectKExtremes # type: ignore
from skfolio.preprocessing import prices_to_returns
from skfolio.prior import BlackLitterman, EmpiricalPrior, FactorModel
from skfolio.uncertainty_set import BootstrapMuUncertaintySet

In [13]:
import pandas as pd

### Loading out asset classes from asset_classes.csv

In [17]:
prices = load_sp500_dataset()
prices.index

DatetimeIndex(['1990-01-02', '1990-01-03', '1990-01-04', '1990-01-05',
               '1990-01-08', '1990-01-09', '1990-01-10', '1990-01-11',
               '1990-01-12', '1990-01-15',
               ...
               '2022-12-14', '2022-12-15', '2022-12-16', '2022-12-19',
               '2022-12-20', '2022-12-21', '2022-12-22', '2022-12-23',
               '2022-12-27', '2022-12-28'],
              dtype='datetime64[ns]', name='Date', length=8313, freq=None)

In [14]:
asset_classes = pd.read_csv('../data/asset_classes.csv')

In [15]:
asset_classes

,Date,AGGG,FXI,HYG,S&P
0,10/11/2024,4.40,33.37,79.57,"5,815.03"
1,10/10/2024,4.40,33.22,79.36,"5,780.05"
2,10/09/2024,4.40,32.79,79.40,"5,792.04"
3,10/08/2024,4.41,33.33,79.45,"5,751.13"
4,10/07/2024,4.41,36.69,79.26,"5,695.94"
...,...,...,...,...,...
1229,10/18/2019,5.17,41.24,87.09,"2,986.20"
1230,10/17/2019,5.17,41.69,87.07,"2,997.90"
1231,10/16/2019,5.15,41.52,86.95,"2,989.70"
1232,10/15/2019,5.15,41.54,87.02,"2,995.70"


train/test split

In [4]:
X = prices_to_returns(prices)
X_train, X_test = train_test_split(X, test_size=0.33, shuffle=False)

Specifying model.

This does some form of convex optimizartion

In [5]:
model = MeanRisk(
     objective_function=ObjectiveFunction.MAXIMIZE_UTILITY,
     risk_measure=RiskMeasure.VARIANCE,
     #risk_aversion= 0.5,
     budget=1.0,
     #max_short=0.2,
     risk_free_rate=0.04
)

In [6]:
model.fit(X_train)

print(model.weights_)

[1.64155266e-01 1.73248584e-10 2.49985932e-11 2.63338588e-01
 1.46572822e-10 5.31129960e-11 3.18275829e-10 1.81842342e-10
 9.31945092e-11 1.43950943e-10 6.54673188e-11 8.15378039e-11
 6.80065528e-02 1.51348506e-10 1.71030366e-10 1.72476868e-10
 1.60200725e-01 3.44298865e-01 1.55983729e-10 1.49875811e-10]


In [7]:
portfolio = model.predict(X_test)

In [8]:
portfolio.composition

,weight
asset,
UNH,0.344299
BBY,0.263339
AAPL,0.164155
RRC,0.160201
MSFT,0.068007


In [9]:
print(portfolio.annualized_sharpe_ratio)
print(portfolio.summary())

0.7710874208227299
Mean                                             0.087%
Annualized Mean                                  21.95%
Variance                                         0.022%
Annualized Variance                               5.42%
Semi-Variance                                    0.011%
Annualized Semi-Variance                          2.67%
Standard Deviation                                1.47%
Annualized Standard Deviation                    23.28%
Semi-Deviation                                    1.03%
Annualized Semi-Deviation                        16.35%
Mean Absolute Deviation                           1.02%
CVaR at 95%                                       3.24%
EVaR at 95%                                       5.81%
Worst Realization                                11.14%
CDaR at 95%                                      23.59%
MAX Drawdown                                     38.44%
Average Drawdown                                  5.75%
EDaR at 95%                  

In [10]:
portfolio.composition

,weight
asset,
UNH,0.344299
BBY,0.263339
AAPL,0.164155
RRC,0.160201
MSFT,0.068007


In [11]:
portfolio.plot_returns()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'hovertemplate': 'variable=returns<br>index=%{x}<br>value=%{y}<extra></extra>',
              'legendgroup': 'returns',
              'line': {'color': '#636efa', 'dash': 'solid'},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': 'returns',
              'showlegend': True,
              'type': 'scattergl',
              'x': array([datetime.datetime(2012, 2, 6, 0, 0),
                          datetime.datetime(2012, 2, 7, 0, 0),
                          datetime.datetime(2012, 2, 8, 0, 0), ...,
                          datetime.datetime(2022, 12, 23, 0, 0),
                          datetime.datetime(2022, 12, 27, 0, 0),
                          datetime.datetime(2022, 12, 28, 0, 0)], dtype=object),
              'xaxis': 'x',
              'y': array([ 0.0145923 ,  0.01834069,  0.00297007, ...,  0.01167511,  0.00021905,
                          -0.0248712 ]),
              'yaxis': 'y'}],
    'layout': {'legend': {'title': {'text': 'variable'}, 'tracegroupgap': 0},
               'margin': {'t': 60},
               'showlegend': False,
               'template': '...',
               'title': {'text': 'Returns'},
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'Observations'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'Returns'}}}
})